## Authoritarian Framework

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate
from langchain_core.prompts.chat import SystemMessage, _convert_to_message
from langchain_core.pydantic_v1 import BaseModel, Field, ValidationError
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.output_parsers import JsonOutputToolsParser, JsonOutputParser

from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_ollama import ChatOllama
# from langchain_community.chat_models import ChatOllama

from langchain_experimental.llms.ollama_functions import OllamaFunctions, parse_response

from langgraph.graph import END, StateGraph, MessageGraph
from langgraph.checkpoint.sqlite import SqliteSaver

import functools
import operator
from typing import List, Sequence, TypedDict, Annotated, Dict, Any, Optional
import json
import os
import random

from IPython.display import Image, display

import concurrent.futures

In [4]:
unique_id = "Authoritarian Optimisation"
os.environ["LANGCHAIN_PROJECT"] = f"Tracing Walkthrough - {unique_id}"

In [5]:
# from langsmith import Client

# client = Client()

In [6]:
class CorePrinciples:
    def __init__(self, core_principles: List[str]):
        self.core_principles = core_principles
    
    def add_principle(self, principle: str):
        """
        Adds a principle to the core principles list.
        
        :param principle: The principle to be added.
        """
        self.core_principles.append(principle)
        
    def __str__(self):
        """
        Returns a string representation of the core principles, each principle is listed on a new line with a preceding dash.
        
        Example:
        - principle 1
        - principle 2
        ...
        """
        return "\n".join([f"- {principle}" for principle in self.core_principles])


class AdvisorAgent:
    """
    Advisor Agent class defining agents that provide feedback on prompts.
    """

    def __init__(self, position: str, core_principles: CorePrinciples, llm = ChatOpenAI(temperature=1.0, model="gpt-4o")):
        self.position = position
        self.core_principles = str(core_principles)
        self.system_message = f"""You are an experienced: {self.position}. Your core principles are:
{self.core_principles}"""
        self.llm = llm

    def review_prompt(self, state: Sequence[BaseMessage], criteria: str):
        """
        Generates a review of the prompt.
        """
        if isinstance(self.llm, ChatOpenAI):
            prompt_text = f"""Your task is to provide feedback on the prompt in the conversation above in light of your core princples.
Always think outside the box and consider unconventional ideas.

Your reviewal process should be as follows:
1. Read the prompt as an experienced: {self.position}. Understand it's content and intent.
2. Explain how you think the prompt can be improved in light of your core principles.
3. Submit your feedback."""
            prompt = ChatPromptTemplate.from_messages(
                [
                    ("system", self.system_message),
                    MessagesPlaceholder(variable_name="messages"),
                    ("system", prompt_text),
                ]
            )
            chain = prompt | self.llm
            result = chain.invoke({"messages": state})
        
    def approval(self, state: Sequence[BaseMessage]):
        """
        Agent to approve or reject the prompt.
        """
        prompt_text = f"""Your task is to decide if the prompt in the conversation above is optimal in light of your core principles.

If you think the prompt is optimal and does not require improvements in light of your core principles, return True.
If you think the prompt needs improvements in light of your core principles, return False.

Your reviewal process should be as follows:
1. Read the prompt as an experienced: {self.position}. Understand it's content and intent.
2. Determine whether the prompt is optimal in light of your core principles.
3. Submit your decision."""
        function_def = {
            "name": "approval",
            "description": "Get approval decision of advisor.",
            "parameters": {
                "type": "object",
                "properties": {
                    "advisor": {"type": "string", "enum": [self.position]},
                    "decision": {"type": "string", "enum": ["True", "False"]},
                },
                "required": ["decision", "advisor"],
            },
        }
        if isinstance(self.llm, ChatOpenAI):
            messages = [
                ("system", self.system_message),
                MessagesPlaceholder(variable_name="messages"),
                ("system", prompt_text),
            ]
            prompt = ChatPromptTemplate.from_messages(messages)
            chain = (
                prompt
                | self.llm.bind_functions(functions=[function_def], function_call="approval")
                | JsonOutputFunctionsParser()
            )
            result = chain.invoke({"messages": state})
            return result

In [7]:
class LeaderAgent:
    """
    LeaderAgent class defining an agent that generates and communicates with advisor agents to help optimise prompts.
    """

    def __init__(self, base_prompt: str, criteria: str = None, advisors: List[AdvisorAgent] = None, llm = ChatOpenAI(temperature=1.0, model="gpt-4o")):
        self.base_prompt = base_prompt
        self.criteria = criteria
        self.system_message = f"""You are an experienced: Lead AI Prompt Engineer. Your core principles are:
- Always pay attention to detail when designing prompts
- Always make informed decisions when designing prompts
- Always be open to new ideas when designing prompts"""
        self.llm = llm
        self.advisors = advisors
        self.iterations = 0

    def reset(self):
        """
        Resets the iterations counter to 0.
        """
        self.iterations = 0
    
    def run_approval(self, state: Sequence[BaseMessage]) -> List[bool]:
        """
        Run the approval process for the prompt. Run concurrently
        """
        with concurrent.futures.ThreadPoolExecutor() as executor:
            futures = [executor.submit(advisor.approval, state) for advisor in self.advisors]
            results = [future.result() for future in concurrent.futures.as_completed(futures)]
        return results
    
    def leader_decision(self, state: Sequence[BaseMessage]):
        """
        LeaderAgent to decide the next advisor or to finish.
        """
        self.iterations += 1
        approval_results = self.run_approval(state)
        print("Approval results:", approval_results)
        # Extract decisions from the results
        if all(str(approval_result['decision']) == "True" for approval_result in approval_results) or self.iterations >= 6:
            self.reset()
            return "FINISH"
        else:
            # Only ask for advise from advisors that disapproved the prompt
            disapproved_advisors = [approval_result['advisor'] for approval_result in approval_results if str(approval_result['decision']) == "False"]
            disapproved_advisors_details = "\n".join([f"{advisor.position}: {advisor.core_principles}" for advisor in self.advisors if advisor.position in disapproved_advisors])
            # shuffle the options to avoid positional bias
            random.shuffle(disapproved_advisors)
            # options = ["FINISH"] + positions
            prompt_text = f"""Your task is to select the next advisor to provide feedback on the prompt in the conversation above.
Think carefully about which advisor will provide the most valuable feedback to make improvements.
            
The success criteria for the prompt are as follows:
{self.criteria}

Select one of the below advisors that disapproved of the previous prompt to provide feedback on how to improve it: 
{disapproved_advisors}

The details of all disapproving advisors and their core principles are as follows: 
{disapproved_advisors_details}

Your selection process should be as follows:
1. Read the prompt as an experienced: Lead AI Prompt Engineer. Understand it's content and intent.
2. Explicitly detail how you think the prompt can be improved. Assume the prompt always needs improvement.
3. Select which disapproving advisor you think will provide the most valuable feedback to make improvements.
4. Submit your selection for the next advisor."""
            function_def = {
                "name": "next",
                "description": "Get the next role.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "next": {"type": "string", "enum": disapproved_advisors},
                    },
                    "required": ["next"],
                },
            }
            prompt = ChatPromptTemplate.from_messages(
                [
                    ("system", self.system_message),
                    MessagesPlaceholder(variable_name="messages"),
                    ("system", prompt_text),
                ]
            )
            if type(self.llm) == ChatOpenAI:
                chain = (
                    prompt
                    | self.llm.bind_functions(functions=[function_def], function_call="next")
                    | JsonOutputFunctionsParser()
                )
                result = chain.invoke({"messages": state})
                return result["next"]

    def update_prompt(self, state: Sequence[BaseMessage]) -> str:
        """
        Updates the prompt with the feedback from the advisor agent.
        """

        if len(state) == 1:
            return {"next": self.leader_decision(state)}
        
        prompt_text = f"""Your task is to improve the prompt in the conversation above in light of your core principles.
If you recieve feedback and recommendations for the prompt, respond with a revised version of your previous attempts actioning the feedback.

The success criteria for the prompt are as follows:
{self.criteria}
You will be penalized if the prompt does not meet this criteria.

Below are strict guidelines that you MUST follow if making changes to the prompt:
- DO NOT modify existing restrictions.
- DO NOT modify or remove negations.
- DO NOT add, modify or remove placeholders denoted by curly braces. If you wish to use curly braces in your response, use double curly braces to avoid confusion with placeholders.
- ALWAYS treat placeholders as the actual content.
You will be penalized if you do not follow these guidelines.

Your update process should be as follows:
1. Read the prompt as an experienced: Head AI Engineer. Understand it's content and intent.
2. Think carefully about how you can implement the most recent feedback and revise the prompt.
3. Explcitly go through each success criteria and ensure the prompt meets them. If not, revise the prompt to make sure it does.
4. Explicitly go through each guideline and ensure the changes adhere to them. If not, revise the prompt to make sure it does.
5. Submit your revised prompt."""
        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", self.system_message),
                MessagesPlaceholder(variable_name="messages"),
                ("system", prompt_text),
            ]
        )
        chain = prompt | self.llm
        result = chain.invoke({"messages": state})
        return {"messages": [AIMessage(content=result.content, name="Leader")], "next": self.leader_decision(state)}

    def construct_advisor_graph(self):
        """
        Constructs a graph of advisor agents based on their roles and functions.
        """

        def advisor_node(state, agent):
            return agent.review_prompt(state["messages"], self.criteria)
        
        def leader_node(state):
            return self.update_prompt(state["messages"]) 
        
        # The agent state is the input to each node in the graph
        class AgentState(TypedDict):
            messages: Annotated[Sequence[BaseMessage], operator.add]
            next: str

        workflow = StateGraph(AgentState)
        for advisor in self.advisors:
            # Create a node for each advisor agent
            node = functools.partial(advisor_node, agent=advisor)
            workflow.add_node(advisor.position, node)
        workflow.add_node("Leader", leader_node)

        members = [advisor.position for advisor in self.advisors]
        for member in members:
            # We want our advisors to ALWAYS "report back" to the leader when done
            workflow.add_edge(member, "Leader")
        # The leader populates the "next" field in the graph state with routes to a node or finishes
        conditional_map = {k: k for k in members}
        conditional_map["FINISH"] = END
        workflow.add_conditional_edges("Leader", lambda x: x["next"], conditional_map)
        workflow.set_entry_point("Leader")

        memory = SqliteSaver.from_conn_string(":memory:")
        graph = workflow.compile(checkpointer=memory)

        return graph
    
    def optimise_prompt(self):
        """
        Optimises a prompt by invoking a graph of advisor agents.
        """
        # Initial state
        initial_state = {
            "messages": [HumanMessage(content=self.base_prompt, name="User")],
        }

        # Construct the graph
        graph = self.construct_advisor_graph()
        # display(Image(graph.get_graph().draw_mermaid_png()))

        n = random.randint(0, 1000)
        config = {
            "configurable": {"thread_id": n},
            "recursion_limit": 50,
            }    

        # Run the graph
        for s in graph.stream(
            initial_state,
            config,
            stream_mode="values",
            ):
            if "__end__" not in s:
                if len(s["messages"]) > 1:
                    s["messages"][-1].pretty_print()
                continue
        
        def message_to_dict(obj):
            if isinstance(obj, HumanMessage) or isinstance(obj, AIMessage):
                return {obj.name: obj.content}
            raise TypeError(f'Object of type {obj.__class__.__name__} is not JSON serializable')

        if type(self.llm) == ChatOpenAI:
            model = self.llm.model_name
        else:
            model = self.llm.model
        temp = int(self.llm.temperature)
        path = f"/Users/iwatson/Documents/Research Project/prompt-optimisation/src/conversations/{model}/conversations_authoritarian_{temp}.json"
        if not os.path.exists(path):
            with open(path, "w") as f:
                json.dump([], f)
        
        with open(path, "r") as f:
            # write messages to json file
            data = json.load(f)
            # get the current key number then increment it
            key = len(data)
            data.append({key: json.dumps(s, default=message_to_dict)})
            
        with open(path, "w") as f:
            json.dump(data, f, indent=4)

        return s


In [8]:
# llm = ChatAnthropic(temperature=0, model="claude-3-haiku-20240307")
# llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")
# llm = ChatOllama(temperature=1, model="mistral:v0.3")
# llm = ChatOllama(temperature=0, model="llama3.1")

In [9]:
from agent_suite import PromptDesignAgents, HumanEvalAgents, GSM8kAgents, SST2Agents

prompt_design_agents = PromptDesignAgents()

style_and_structure_expert = AdvisorAgent("Style_and_Structure_Expert", CorePrinciples(prompt_design_agents.get_style_and_structure_principles()), llm)
conciseness_and_clarity_expert = AdvisorAgent("Conciseness_and_Clarity_Expert", CorePrinciples(prompt_design_agents.get_conciseness_and_clarity_principles()), llm)
contextual_relevance_expert = AdvisorAgent("Contextual_Relevance_Expert", CorePrinciples(prompt_design_agents.get_contextual_relevance_principles()), llm)
task_alignment_expert = AdvisorAgent("Task_Alignment_Expert", CorePrinciples(prompt_design_agents.get_task_alignment_principles()), llm)
example_demonstration_expert = AdvisorAgent("Example_Demonstration_Expert", CorePrinciples(prompt_design_agents.get_example_demonstration_principles()), llm)
incremental_prompting_expert = AdvisorAgent("Incremental_Prompting_Expert", CorePrinciples(prompt_design_agents.get_incremental_prompting_principles()), llm)

human_eval_agents = HumanEvalAgents()

code_reviewer = AdvisorAgent("Code_Reviewer", CorePrinciples(human_eval_agents.get_code_reviewer_principles()), llm)
software_engineer = AdvisorAgent("Software_Engineer", CorePrinciples(human_eval_agents.get_software_engineering_principles()), llm)
software_architect = AdvisorAgent("Software_Architect", CorePrinciples(human_eval_agents.get_software_architecture_principles()), llm)

gsm8k_agents = GSM8kAgents()

mathematician = AdvisorAgent("Mathematician", CorePrinciples(gsm8k_agents.get_mathematician_principles()), llm)
word_problem_solver = AdvisorAgent("Word_Problem_Solver", CorePrinciples(gsm8k_agents.get_word_problem_solver_principles()), llm)

sst2_agents = SST2Agents()

graded_sentiment_analyst = AdvisorAgent("Graded_Sentiment_Analyst", CorePrinciples(sst2_agents.get_graded_sentiment_analyst_principles()), llm)
emotive_sentiment_analyst = AdvisorAgent("Emotive_Sentiment_Analyst", CorePrinciples(sst2_agents.get_emotive_sentiment_analyst_principles()), llm)
aspect_based_sentiment_analyst = AdvisorAgent("Aspect_Based_Sentiment_Analyst", CorePrinciples(sst2_agents.get_aspect_based_sentiment_analyst_principles()), llm)

In [10]:
from prompts.gpt_4o.human_eval_prompts import HumanEvalPrompts
from prompts.gpt_4o.gsm8k_prompts import GSM8KPrompts
from prompts.gpt_4o.sst2_prompts import SST2Prompts

human_eval_prompts = HumanEvalPrompts()
gsm8k_prompts = GSM8KPrompts()
sst2_prompts = SST2Prompts()

baseline_prompt = sst2_prompts.get_baseline_prompt()
criteria = sst2_prompts.get_criteria()

leader_agent = LeaderAgent(
    base_prompt=baseline_prompt,
    criteria=criteria,
    advisors=[
        style_and_structure_expert,
        conciseness_and_clarity_expert,
        contextual_relevance_expert,
        task_alignment_expert,
        example_demonstration_expert,
        incremental_prompting_expert,
        graded_sentiment_analyst,
        emotive_sentiment_analyst,
        aspect_based_sentiment_analyst,
    ],
    llm = llm
)
for advisor in leader_agent.advisors:
    print("Position: ", advisor.position + "\nCore Principles:\n", advisor.core_principles)

Position:  Style_and_Structure_Expert
Core Principles:
 - Always structure prompts logically for the task
- Always use a style and tone in prompts that is appropriate for the task
- Always assign a role to the language model that is relevant to the task
Position:  Conciseness_and_Clarity_Expert
Core Principles:
 - Always write clear and concise prompts
- Always use simple and direct language in prompts
- Always avoid ambiguity in prompts
Position:  Contextual_Relevance_Expert
Core Principles:
 - Always provide context to help the model understand the task
- Always write prompts informed by the context of the task
- Always design contextually relevant roles for the language model
Position:  Task_Alignment_Expert
Core Principles:
 - Always write prompts that align with the task criteria
- Always tailor instructions to the task to guide the model
- Always make the task abundantly clear to the model in the prompt
Position:  Example_Demonstration_Expert
Core Principles:
 - Always provide ex

In [11]:
import time

times = []
for _ in range(1):
    start = time.time()
    result = leader_agent.optimise_prompt()
    end = time.time()
    times.append(end - start)
    result["messages"][-1].pretty_print()
    print("----")

Approval results: [{'advisor': 'Incremental_Prompting_Expert', 'decision': 'False'}, {'advisor': 'Contextual_Relevance_Expert', 'decision': 'False'}, {'advisor': 'Graded_Sentiment_Analyst', 'decision': 'False'}, {'advisor': 'Task_Alignment_Expert', 'decision': 'False'}, {'advisor': 'Emotive_Sentiment_Analyst', 'decision': 'False'}, {'advisor': 'Example_Demonstration_Expert', 'decision': 'False'}, {'advisor': 'Style_and_Structure_Expert', 'decision': 'False'}, {'advisor': 'Conciseness_and_Clarity_Expert', 'decision': 'False'}, {'advisor': 'Aspect_Based_Sentiment_Analyst', 'decision': 'False'}]
Approval results: [{'advisor': 'Style_and_Structure_Expert', 'decision': 'False'}, {'advisor': 'Task_Alignment_Expert', 'decision': 'False'}, {'advisor': 'Contextual_Relevance_Expert', 'decision': 'False'}, {'advisor': 'Emotive_Sentiment_Analyst', 'decision': 'False'}, {'advisor': 'Graded_Sentiment_Analyst', 'decision': 'False'}, {'advisor': 'Incremental_Prompting_Expert', 'decision': 'False'}, {'

FileNotFoundError: [Errno 2] No such file or directory: '/Users/iwatson/Documents/Research Project/prompt-optimisation/src/conversations/gpt-4o-mini/conversations_authoritarian_0.json'

In [ ]:
print("Max time: ", max(times))
print("Min time: ", min(times))
print("Average time: ", sum(times) / len(times))

ValueError: max() arg is an empty sequence